# Security Constrained OPF (SCOPF)

Finds least-cost dispatch satisfying base-case and N-1 constraints.

Import the case and instantiate the `GridWorkBench`.

```python
from esapp import GridWorkBench
from esapp.components import *

wb = GridWorkBench(case_path)
```

In [ ]:
# This cell is hidden in the documentation.
from esapp import GridWorkBench
from esapp.components import *
import matplotlib.pyplot as plt
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_dual_bar

## Setup SCOPF Optimization

Initialize the solver and prepare contingency constraints for the security-constrained problem:

The Primal LP solver is PowerWorld's optimization engine. Auto-insert N-1 contingencies to make the optimization security-constrained:

In [ ]:
# Capture pre-OPF dispatch
pre_opf = wb[Gen, ['BusNum', 'GenMW', 'GenStatus']]
pre_opf_online = pre_opf[pre_opf['GenStatus'] == 'Closed'].copy()

wb.esa.InitializePrimalLP()
wb.auto_insert_contingencies()

## Solve SCOPF

In [ ]:
wb.esa.SolveFullSCOPF()

production_cost = wb[Area, "GenProdCost"]
print("Production Cost by Area:")
print(production_cost.to_string(index=False))

In [ ]:
post_opf = wb[Gen, ['BusNum', 'GenMW', 'GenStatus']]
post_opf_online = post_opf[post_opf['GenStatus'] == 'Closed'].copy()

plot_dual_bar(pre_opf_online['GenMW'].values, post_opf_online['GenMW'].values,
              label_a='Pre-OPF', label_b='Post-OPF',
              xlabel='Generator Index', ylabel='MW Output',
              title='Generator Dispatch: Pre-OPF vs Post-SCOPF')